# First Prototype

**Implemented Features:**
* Nodes
* Artifacts
* Processes
* Edges (used, wasGeneratedBy) between Processes and Artifacts
* Edges (triggered) between Processes
* decorator API for graph creation via @optex_process and @optex_composition
* Implicit graph Accounts -- A Process node may contain some child_processes. These child_processes are Processes which are scoped within the original Node

**Not Implemented:**
* Agents
* Graph registration or anything with merging
* Roles
* Annotations

### TODO in this prototype:
* Edges (derivedFrom) between Artifacts. This will be done via a context manager.
* Better display of results. It's hard to read this code and understand how it corresponds to graph structure

### Feedback Requested
How is the graph structure? Is it easy to navigate programmatically? Probably not! What alternatives are there?

In [151]:
from __future__ import annotations
from typing import Sequence, Mapping, Any, Callable, Tuple
import dataclasses
import enum
import numpy as np
import pandas as pd
from functools import wraps

        
class Node():
    def __init__(self):
        self.parents = []
        self.children = []
        self.agents = []


class Artifact(Node):
    def __init__(self, data):
        super().__init__()
        self.data = data
        self.entered_scope = None
        self.name = 'default'
        
    def add_child(self, child):
        if not self.entered_scope:
            self.children.append(child)
        else:
            self.entered_scope.add_child_process(child)
    
    def add_parent(self, parent):
        self.parents.append(parent)
    
    def enter_scope(self, process):
        self.entered_scope = process
    
    def leave_scope(self, process):
        self.entered_scope = None
        self.parents[0].add_child_process(process)
        self.parents[0].remove_child(self)
        self.parents = []
        
    def __str__(self):
        return self.name


class Process(Node):
    def __init__(self, transformation):
        super().__init__()
        self.transformation = transformation
        self.child_processes = []

    def add_child(self, child):
        self.children.append(child)

    def add_parent(self, parent):
        if not parent.entered_scope:
            self.parents.append(parent)
            
    def add_child_process(self, process):
        self.child_processes.append(process)
        
    def remove_child(self, child):
        self.children = list(filter(lambda c: c != child, self.children))
        
    def __str__(self):
        return str(self.transformation)


def optex_composition(transformation):
    @wraps(transformation)
    def wrapped_function(*artifact_args, **artifact_kwargs):
        
        process = Process(transformation)

        for arg in artifact_args:
            arg.add_child(process)
            process.add_parent(arg)
            arg.enter_scope(process)

        for k, v in artifact_kwargs.items():
            v.add_child(process)
            process.add_parent(v)
            v.enter_scope(process)

        results = transformation(*artifact_args, **artifact_kwargs)

        if isinstance(results, Tuple):
            artifact_results = []
            for artifact in results:
                artifact.leave_scope(process)
                artifact.add_parent(process)
                artifact_results.append(artifact)
                process.add_child(artifact)
            return artifact_results
        else:
            results.leave_scope(process)
            results.add_parent(process)
            process.add_child(results)
            return results
    return wrapped_function


def optex_process(transformation):
    @wraps(transformation)
    def wrapped_function(*artifact_args, **artifact_kwargs):
        args = []
        kwargs = {}
        
        process = Process(transformation)

        for arg in artifact_args:
            arg.add_child(process)
            args.append(arg.data)
            process.add_parent(arg)

        for k, v in artifact_kwargs.items():
            v.add_child(process)
            kwargs[k] = v.data
            process.add_parent(v)

        results = transformation(*args, **kwargs)

        if isinstance(results, Tuple):
            artifact_results = []
            for result in results:
                artifact = Artifact(result)
                artifact.add_parent(process)
                artifact_results.append(artifact)
                process.add_child(artifact)
            return artifact_results
        else:
            artifact = Artifact(results)
            artifact.add_parent(process)
            process.add_child(artifact)
            return artifact
    return wrapped_function

In [159]:
@optex_process
def load_data(dataset_name):
    if dataset_name == "dataset_one":
        return pd.DataFrame({
            'a': [1,2,3,4,5,2,4],
            'b': [1,1,6,32,5,1,3],
            'fk': [1,1,0,1,0,0,0]
        })
    elif dataset_name == "dataset_two":
        return pd.DataFrame({
            'fk': [1,0],
            'x': [10,2],
            'y': [0,4],
            'z': [34,199]
        })
    else:
        raise Exception("Dataset not recognized!")


@optex_process
def join_datasets(df1, df2):
    return pd.merge(df1, df2, on="fk")


@optex_process
def standardize(df):
    return (df - df.mean()) / df.std()


class D1Pipeline:
    def __init__(self):
        pass
    
    @staticmethod
    @optex_process
    def transform(df):
        df = df.copy()
        df['a'] = df['a'] + 10
        return df


class D2Pipeline:
    def __init__(self):
        pass
    
    @staticmethod
    @optex_process
    def transform(df):
        fk = df['fk'].copy()
        # df = standardize(df.drop('fk', axis=1))
        df = (df - df.mean()) / df.std()
        df['fk'] = fk
        return df


class JoinedPipeline:
    def __init__(self):
        pass
    
    @staticmethod
    @optex_process
    def transform(df):
        df = df.drop('fk', axis=1)
        df['ax'] = df['a'] + df['x']
        df['by'] = df['b'] + df['y']
        return df.drop(columns=['a', 'b', 'x', 'y'])


class Pipeline:
    def __init__(self, d1_name, d2_name):
        self.dataset1 = load_data(Artifact(d1_name))
        self.dataset1.name = 'd1'
        self.dataset2 = load_data(Artifact(d2_name))
        self.dataset2.name = 'd2'
    
    def run_pipline(self):
        return Pipeline.transform(self.dataset1, self.dataset2)

    @staticmethod
    @optex_composition
    def transform(d1, d2):
        d1 = D1Pipeline.transform(d1)
        d1.name = 'd1p_out'
        d2 = D2Pipeline.transform(d2)
        d2.name = 'd2p_out'
        df = join_datasets(d1, d2)
        df.name = 'joined'
        final = JoinedPipeline.transform(df)
        final.name = 'final'
        return final

In [160]:
x = Pipeline("dataset_one", "dataset_two").run_pipline()

In [161]:
str(x)

'final'

In [162]:
str(x.parents[0])

'<function Pipeline.transform at 0x7f56aa633af0>'

In [163]:
list(map(str, x.parents[0].parents))

['d1', 'd2']

In [169]:
str(x.parents[0].parents[0].parents[0].parents[0].parents)

'[]'

In [194]:
str(x.parents[0].child_processes[0].children[0].children[0].parents[1])

'd2p_out'